<a href="https://colab.research.google.com/github/afortuny/DeepLearningProduction/blob/main/Image_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#hide 
!pip install -Uqq fastbook # remember to restart runtime every time you run this line
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [4]:
! pip install simple_image_download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from simple_image_download import simple_image_download as simp



In [6]:
from simple_image_download import simple_image_download as simp 
response = simp.simple_image_download

In [7]:
lst=["adidas running shoes"]

In [8]:
for rep in lst:
 response().download(rep , 300)

In [9]:
dls = ImageDataLoaders.from_folder(Path('/content/simple_images'),valid_pct=0.2)

In [10]:
dls.valid_ds.items[:3]

[Path('/content/simple_images/adidas running shoes/adidas running shoes_63.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_292.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_246.jpg')]

In [11]:
def get_dls(size, bs, workers=None):
    source = Path('/content/simple_images')
    
    def dummy_label(o): return 0
    files = get_image_files(source)
    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(size, min_scale=0.9)], [dummy_label]]
    
    dsets = Datasets(files, tfms=tfms, splits=RandomSplitter(valid_pct=0.1)(files))
    
    batch_tfms = [IntToFloatTensor]
    dls = dsets.dataloaders(bs=bs, num_workers=workers, after_batch=batch_tfms, device='cpu')
    return dls

In [12]:
bs,resize,size = 64,160,224
dls = get_dls(resize, bs)

In [ ]:
!pip install self-supervised

In [16]:
from self_supervised.layers import *

In [17]:
from self_supervised.vision.moco import *

In [25]:
from PIL import Image
import glob

image_list = []
resized_images = []

for filename in glob.glob('/content/simple_images/adidas running shoes\\*.jpg'):
    print(filename)
    img = Image.open(filename)
    image_list.append(img)

for image in image_list:
    image = image.resize((128, 128))
    resized_images.append(image)

for (i, new) in enumerate(resized_images):
    new.save('{}{}{}'.format('/content/simple_images/adidas running shoes\\', i+1, '.jpg'))

In [25]:
encoder = create_encoder("tf_efficientnet_b4_ns", n_in=3, pretrained=False) # a timm encoder
model = create_moco_model(encoder, hidden_size=2048, projection_size=128)
aug_pipelines = get_moco_aug_pipelines(size=size)
learn = Learner(new_dl, model,loss_func=noop,cbs=[MOCO(aug_pipelines=aug_pipelines, K=128)])
learn.fit_flat_cos(100, 1e-2)

IndexError: ignored

In [24]:
new_dl = dls.test_dl(dls.valid_ds.items[:], with_labels=True)

In [23]:
valid_lst[1]

Path('/content/simple_images/adidas running shoes/adidas running shoes_246.jpg')

In [ ]:
verify_images()

In [19]:
from google.colab import files
files.download('/content/simple_images/adidas running shoes')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>